In [70]:
# Necessary imports

import numpy as np
import pandas as pd
import timeit

In [71]:
# List of files that contains datasets

file_names = ["HW2_2022402150_10.txt",
              "HW2_2022402150_100.txt",
              "HW2_2022402150_1000.txt"]

In [72]:
# Calculates mean square error of two arrays

def mse(base_arr, compare_arr):
    return np.sum(np.square(base_arr-compare_arr))/len(base_arr)

In [73]:
# Returns array from a dataset in a file

def pull_file(file_name):
    return np.genfromtxt(file_name, delimiter=" ")

In [74]:
# Returns steady state probabilites by utilizing eigenvalues and eigenvectors

def method_analytic_solution(arr):
    eigenvalues, eigenvectors = np.linalg.eig(arr.T) # Returns eigenvalues and left eigenvectors
    left_eigenvector = eigenvectors[:, np.where(np.isclose(eigenvalues, 1))[0][0]].T # Finds the eigenvector which has the approximate eigenvalue of 1
    left_eigenvector /= np.sum(left_eigenvector) # Normalizes that vector
    return np.real(left_eigenvector)

In [75]:
# Returns steady state probabilities by executing a simulation

def method_simulation(arr, T=1000000):
    count = 0
    arr_len = len(arr[0]) # Returns the lenght of a row of the array
    realizations = {i:0 for i in range(arr_len)} # Creates a dictionary that has the necessary keys
    current_row = arr[np.random.randint(0, arr_len)] # Chooses a random row
    while count < T:
        r = np.random.random() # Chooses a random number between 0 and 1
        for c, entry in enumerate(current_row): # To find which interval consists r in the current row
            r -= entry # Decreases r by the entry
            if r <= 0: # Where r reaches 0, it is the interval that consists r
                break
        realizations[c] += 1 # Add visit to realizations
        current_row = arr[c] # Go next row
        count += 1
    return np.array(list(realizations.values()))/T

In [76]:
# Multiplies the array by itself until its column starts to imitate one another

def method_matrix_multiplication(arr, tolerance=0.000001):
    arr_len = len(arr[0])
    not_done = True
    while not_done: # Executes until stop statement is true  
        for c in range(arr_len):      
            error = False
            column = arr[:, c] # Takes the transpose
            col_avg = np.sum(column)/arr_len # Finds the average of the transposed column
            for entry in column:
                if np.abs(entry- col_avg) >= tolerance: # Checks whether the tolerance is violated
                    error = True # If any of the values is greater than the tolerance, raises error
                    break
            if error: # If there is an error, breaks from the loop and starts to next one
                break    
        else:
            not_done = False # If no errors is encountered, then while loop can be broken
             
        if not_done:         
            arr = np.matmul(arr, arr) # If there is an error, multiplies the array by itself to produce the next array for the next loop
                  
    return arr[0] # Returned row does not matter since all the rows are approxiametly identically same

In [88]:
# Measures the runtimes of each method 10 times and returns their mean

def measure_time(arr):
    
    exe_time_1 = timeit.timeit(lambda: method_analytic_solution(arr), number=10)
    exe_time_2 = timeit.timeit(lambda: method_simulation(arr), number=10)
    exe_time_3 = timeit.timeit(lambda: method_matrix_multiplication(arr), number=10)
    
    return exe_time_1, exe_time_2, exe_time_3

In [90]:
# Returns the final dataframe by using previous declared functions

def create_dataframe(file_names):
    datasets = {} # Stores the information to create data frame in further steps

    for  n, dataset in enumerate(file_names): # Loops over all files
        arr = pull_file(dataset) # Creates an array from the current file

        # Creates the variables that stores steady states
        analytic_sol = method_analytic_solution(arr)
        simulation_sol = method_simulation(arr)
        multiplication_sol = method_matrix_multiplication(arr)
        
        # Creates the variables that stores runtimes
        runtime_1, runtime_2, runtime_3 = measure_time(arr)

        # Creates a dictionary that stores the information of the current array
        df_dict = {(f"Dataset {n+1}", "MSE"): ["-", mse(analytic_sol, simulation_sol), mse(analytic_sol, multiplication_sol)],
                   (f"Dataset {n+1}", "Runtime (s)"): [runtime_1, runtime_2, runtime_3]}

        # Extends the "datasets" dictionary with freshly created dictionary
        datasets.update(df_dict)


    method_indexes = ["Method 1", "Method 2", "Method 3"] # To index row headers
    df = pd.DataFrame(data=datasets) # Creates a dataframe
    df.index = method_indexes  # Indexes rows
    return df # Returns the final data frame

In [79]:
# Creates an absorbing state for a given array

def create_absorbing_state(arr):
    arr_len = len(arr[0]) # Finds the lenght of a row of given array
    arr[0] = np.array([1] + [0 for i in range(arr_len-1)]) # Changes the first row with 0's except the first entry which is changed with 1
    return arr # Returns the new array

In [80]:
# For given array with an absorbing state, returns their steady states

def check_absorbing_state(arr):
    return method_analytic_solution(arr), method_simulation(arr), method_matrix_multiplication(arr)

In [103]:
# Returns step counts that executed in method 3 for each case

def number_of_iterations_of_method_3(file_names):
    steps = {}
    
    # Change the function to return step count
    def method_matrix_multiplication_step_count(arr, tolerance=0.000001):
        arr_len = len(arr[0])
        not_done = True
        step = 0
        while not_done:
            for c in range(arr_len):      
                error = False
                column = arr[:, c]
                col_avg = np.sum(column)/arr_len
                for entry in column:
                    if np.abs(entry- col_avg) >= tolerance:  
                        error = True 
                        break
                if error: 
                    break    
            else:
                not_done = False 
                
            if not_done:         
                arr = np.matmul(arr, arr) 
                step += 1
                
        return step
    
    for n, file_name in enumerate(file_names):
        steps[f"Dataset {n+1}"] = method_matrix_multiplication_step_count(pull_file(file_name))
        
    return steps

In [93]:
# Output the data frame

create_dataframe(file_names)

Dataset 1             Dataset 2             Dataset 3            
               MSE Runtime (s)       MSE Runtime (s)       MSE Runtime (s)
Method 1         -    0.001040         -    0.067954         -    7.901031
Method 2       0.0   16.286761       0.0   74.319093       0.0  680.653591
Method 3       0.0    0.002049       0.0    0.092203       0.0    8.429134

My expectations and results are coincide, it was clear that mean square error is near zero since matrix multiplication has the sensivity of 0.000001 and simulation runs for a million step. Those are the reasons that functions output such a clean steady states that are almost exactly as the same output of analytical solution. However the exact MSE's are not zero as the data frame shows but a value is so small in the order of e^-7 that rounds to zero. Method 2 and Method 3 are both reliable but it depends on the tolerance of the method 3 and how many simulations will be executed by the method 2. One can make them very reliable by making their precision higher but if I were to choose a more reliable method, I would choose method 3 from 2 and 3 because of the cost of precision comes as runtime and this brings us to examination of the runtimes. 


It can be easily seen that simulation method is by far the most time consuming method of them all since it simulates all the steps until it executes for a million time. Method Matrix Multiplication and Method Analtyic Solution is very close for the lenght of runtimes but since Analtyic Solution directly calculates the eigenvalues and eigenvectors, it comes out victorius because of the Method Matrix Multiplication has to do great number of multiplications even tough the number of multiplications is very low. Let us examine the number of steps required for the method 3 by introducing a new function.


In [104]:
# Returns a dictionary of number of steps that are executed for each dataset

number_of_iterations_of_method_3(file_names)

{'Dataset 1': 4, 'Dataset 2': 3, 'Dataset 3': 2}

It can be seen that number of iterations may differ for each dataset even though number of iterations may be small, the required number of calculations are exponentially geting bigger that is the reason why runtime expands.

In [106]:
# An absorbing state is added to dataset 1

arr_absorbing = create_absorbing_state(pull_file(file_names[0]))

In [109]:
for sol in check_absorbing_state(arr_absorbing):
    print(sol)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[9.99995e-01 0.00000e+00 2.00000e-06 0.00000e+00 0.00000e+00 1.00000e-06
 1.00000e-06 0.00000e+00 0.00000e+00 1.00000e-06]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Steady state solutions show that in the long run markov chain is always on the state 0 since it become an absorbing state.

Note: The steady states of datasets can be rounded to [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]